# Thresholding on Image

## Introduction

Thresholding is an image processing technique used to separate features of interest from the background in an image. it involves setting a threshold value, which acts as a boundary to determine whether a pixel's intensity or color value is considered part of the object or background. we aim to use Thresholds to enhance the quality of images before performing object detection which prepares images for better object detection and classification performance.

Once objects are segmented using thresholding, various algorithms can be applied to detect and recognize objects based on their shape, size, texture, or other visual features.

There are many types of Thresholding in the context of Image Processing, where we will use the most famous types (THRESH_BINARY, THRESH_OTSU, ADAPTIVE_THRESH where the last type can apply with two ways: thresh_arithmetic_mean and thresh_gaussian_mean)

### 1. THRESH_BINARY


It is a manual thresholding technique where the threshold value is specified by the user, then it works by setting all pixel values below a specified threshold to 0 and all pixel values equal to or above the threshold to a maximum value (which also specified by the user (usually 255)).

$$
dst(x, y) = \begin{cases} maxValue, & \text{if } src(x, y) \geq threshold \\ 0, & \text{if } src(x, y) < threshold \end{cases}
$$

This type of Thresholding is commonly used when there is a clear distinction between foreground and background intensities in the image.


### 2. THRESH_OTSU


It is an automatic thresholding technique that selects the optimal threshold value based on the image histogram, Otsu's method assumes that the image contains two classes of pixels: foreground and background. it iteratively calculates the threshold value that minimizes the intra-class variance within each class and maximizes the inter-class variance between the classes.

Mathematically, THRESH_OTSU calculates the threshold value using a technique called the "maximum between-class variance" method:

$$
\sigma_B^2(T) = \omega_0(T) \cdot \omega_1(T) \cdot (\mu_0(T) - \mu_1(T))^2
$$

Where:

- $P(i):$ is the normalized histogram of the image, representing the probability of intensity level $i$ occurring in the image.
- $L:$ is the total number of intensity levels in the image.
- $\omega_0(T) = \sum_{i=0}^{T} P(i):$ is the probability of the background class up to threshold $T$.
- $\omega_1(T) = \sum_{i=T+1}^{L-1} P(i):$ is the probability of the foreground class starting from threshold $T+1$.
- $\mu_0(T) = \sum_{i=0}^{T} \frac{i \cdot P(i)}{\omega_0(T)}:$ is the mean intensity of the background class up to threshold $T$.
- $\mu_1(T) = \sum_{i=T+1}^{L-1} \frac{i \cdot P(i)}{\omega_1(T)}:$ is the mean intensity of the foreground class starting from threshold $T+1$.

This type of Thresholding is commonly used when to determine automatically the optimal threshold for image segmentation based on the distribution of pixel intensities in the image.


### 3. ADAPTIVE_THRESH

Adaptive thresholding techniques such as THRESH_MEAN_C and THRESH_GAUSSIAN_C dynamically adjust the threshold value for each pixel based on the local neighborhood information.


#### 3-1. THRESH_MEAN_C (Arithmetic Mean)


computes the threshold for each pixel based on the arithmetic mean intensity of the neighboring pixels within a specified window around it.

Mathematically, the threshold value for pixel (x, y) is determined as:

$$
T(x, y) = \frac{1}{N} \sum_{(x', y') \in \text{neighborhood of } (x, y)} I(x', y')
$$

Where:

- $N:$ is the total number of pixels in the neighborhood,
- $I(x', y'):$ is the intensity of pixel \( (x', y') \) in the neighborhood.


#### 3-2. THRESH_GAUSSIAN_C (Gaussian Mean)


computes the threshold for each pixel based on the weighted average (convolution) of the intensities of neighboring pixels within a specified window around it, using a Gaussian kernel.

Mathematically, the threshold value for pixel (x, y) is determined as:

$$
T(x, y) = \sum_{(x', y') \in \text{neighborhood of } (x, y)} I(x', y') \cdot G(x-x', y-y')
$$

Where:

- $I(x', y'):$ Represents the intensity or value of the pixel at position (x', y') in the neighborhood.
- $G(x-x', y-y'):$ Represents the weight or contribution of the pixel at position (x', y') in the neighborhood to the threshold value at pixel (x, y) (This weight is often determined by a Gaussian kernel, where the closer the pixel is to the center (x, y), the higher the weight it receives. This helps in giving more importance to the central pixels in the neighborhood).

This type of Thresholding (ADAPTIVE_THRESH) is effective for segmenting images with varying lighting conditions and local image characteristics


## Implementation

In [ ]:
from expt_utils import *

In [ ]:
def thresh_bin(img_gray, thresh=None):
    if thresh is None:
        thresh = np.mean(img_gray)
    img_bin = cv.threshold(img_gray, thresh, 255, cv.THRESH_BINARY)[1]
    return img_bin


def thresh_bin_inv(img_gray, thresh=None):
    if thresh is None:
        thresh = np.mean(img_gray)
    img_bin = cv.threshold(img_gray, thresh, 255, cv.THRESH_BINARY_INV)[1]
    return img_bin


def thresh_otsu(img_gray):
    img_bin = cv.threshold(
        img_gray, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)[1]
    return img_bin


def thresh_otsu_inv(img_gray):
    img_bin = cv.threshold(
        img_gray, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)[1]
    return img_bin


def thresh_adapt_mean(img_gray):
    img_bin = cv.adaptiveThreshold(
        img_gray, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 11, 2)
    return img_bin


def thresh_adapt_gaussian(img_gray):
    img_bin = cv.adaptiveThreshold(
        img_gray, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 11, 2)
    return img_bin

In [ ]:
img = plt.imread(f'{DS_DIR}/frames/train/00049/00049_3880.jpg')
img_gray = cv.cvtColor(img, cv.COLOR_RGB2GRAY)


def _render(thresh):
    img_bin_inv = thresh_bin_inv(img_gray, thresh)
    cv.imshow('Binary Image', img_bin_inv)


_render(thresh=127)
cv.createTrackbar('Threshold', 'Binary Image', 127, 255, _render)

destroy_when_esc()

In [ ]:
img_bin_inv = thresh_bin_inv(img_gray)
img_bin_otsu_inv = thresh_otsu_inv(img_gray)
img_bin_adapt_mean = thresh_adapt_mean(img_gray)
img_bin_adapt_gaussian = thresh_adapt_gaussian(img_gray)

plt.figure(figsize=(10, 10), tight_layout=True)

plt.subplot(3, 2, 1), plt.axis('off'), plt.title('Image')
plt.imshow(img)

plt.subplot(3, 2, 2), plt.axis('off'), plt.title('Gray Image')
plt.imshow(img_gray, cmap='gray')

plt.subplot(3, 2, 3), plt.axis('off'), plt.title('Binary Threshold (inv) (thresh=127)')
plt.imshow(img_bin_inv, cmap='gray')

plt.subplot(3, 2, 4), plt.axis('off'), plt.title('Otsu Threshold (inv)')
plt.imshow(img_bin_otsu_inv, cmap='gray')

plt.subplot(3, 2, 5), plt.axis('off'), plt.title('Adaptive Threshold (mean)')
plt.imshow(img_bin_adapt_mean, cmap='gray')

plt.subplot(3, 2, 6), plt.axis('off'), plt.title('Adaptive Threshold (gussian)')
plt.imshow(img_bin_adapt_gaussian, cmap='gray')

plt.savefig(f'{OUT_DIR}/02-01-threshold_types')

### Compare and Evaluate the filters

In [ ]:
img_ground_truth = cv.imread(f'{IMG_DIR}/ground_truth/00049_3880.jpg', cv.IMREAD_GRAYSCALE)


pd.DataFrame([
    evaluate(img_bin_inv, img_ground_truth),
    evaluate(img_bin_otsu_inv, img_ground_truth),
    evaluate(img_bin_adapt_mean, img_ground_truth),
    evaluate(img_bin_adapt_gaussian, img_ground_truth),
],
    index=[
        'Binary Thresholding Inverse (127)',
        'Otsu Thresholding Inberse',
        'Adaptive THresholding (mean)',
        'Adaptive THresholding (gaussian)',
])

## Conclusion

We determine visually (what is the best clarity for an object) that adaptive threshold (GAUSSIAN_C) is best type of other thresholding types that used (thresh_binary, thresh_otsu)